Для решения задачи я использовал парсер. Для парсинга слов, которые относятся к числам, я переделал немного бибилиотеку с гитхаба: https://github.com/SergeyShk/Word-to-Number-Russian

Библиотека без ошибок находит слова в исходных данных, которы относятся к числам и группирует подряд идущие числа в группы. Но без тюнинга алгоритма, который решает как преобразовать группу слов в одно или несколько чисел, получалось качество 0.84 на тестовых данных, соответсвенно я итеративно улучшал этот алгоритм чтобы покрыть как можно больше случаев, тем самым увеличив качество до 0.96. Однако остались случаи, которые довольно сложно решить моим парсером

Сам алгоритм работает следующим образом:
Для каждой группы слов добовляем первое число на стек, если следующее число имеет меньшую разрядность то мы добавляем его к числу которое находится на верху стека, если разрядность такая же то добавляем число на стек. Разрядность я проверяю с помощью ```len(str(top)) > len(str(cur_num))``` правда это не всегда хорошо работает, например в этом случае: семьдесят миллионов два миллиона



## Установка библиотек

In [1]:
import pandas as pd

In [2]:
!pip install natasha==0.10.0

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.9/777.9 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 80.9 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=f766c839431944bef5c5fa8481ec13a89a93eedea7dcb7d983edef834d476a71
  Stored in directory: /root/.cache/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built docopt


In [3]:
!pip install yargy==0.12.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 26.8 MB/s eta 0:00:00
  Attempting uninstall: pymorphy2
    Found existing installation: pymorphy2 0.9.1
    Uninstalling pymorphy2-0.9.1:
      Successfully uninstalled pymorphy2-0.9.1
  Attempting uninstall: yargy
    Found existing installation: yargy 0.16.0
    Uninstalling yargy-0.16.0:
      Successfully uninstalled yargy-0.16.0


## Решение

### Правила для парсера

In [5]:
# тут я почти взял почти как есть правила для парсера, по которым
# он будет находить слова, относящиеся в числам

from yargy import rule, or_
from yargy.pipelines import morph_pipeline, caseless_pipeline
from yargy.interpretation import fact, const
from yargy.predicates import eq, caseless, normalized, type

Number = fact('Number', ['int', 'multiplier'])
NUMS_RAW = {
    'ноль': 0,
    'нуль': 0,
    'один': 1,
    'два': 2,
    'двое': 2,
    'три': 3,
    'четыре': 4,
    'пять': 5,
    'шесть': 6,
    'семь': 7,
    'восемь': 8,
    'девять': 9,
    'десять': 10,
    'одиннадцать': 11,
    'двенадцать': 12,
    'тринадцать': 13,
    'четырнадцать': 14,
    'пятнадцать': 15,
    'шестнадцать': 16,
    'семнадцать': 17,
    'восемнадцать': 18,
    'девятнадцать': 19,
    'двадцать': 20,
    'тридцать': 30,
    'сорок': 40,
    'пятьдесят': 50,
    'шестьдесят': 60,
    'семьдесят': 70,
    'восемьдесят': 80,
    'девяносто': 90,
    'сто': 100,
    'двести': 200,
    'двеси': 200,
    'дваста': 200,
    'триста': 300,
    'четыреста': 400,
    'пятьсот': 500,
    'шестьсот': 600,
    'семьсот': 700,
    'восемьсот': 800,
    'девятьсот': 900,
    'тысяча': 10**3,
    'тыщи': 10**3,
    'дветысячи': 2*10**3,
    'миллион': 10**6,
    'миллиард': 10**9,
    'триллион': 10**12,
}

DOT = eq('.')
INT = type('INT')

THOUSANDTH = rule(caseless_pipeline(['тысячных', 'тысячная'])).interpretation(const(10**-3))
HUNDREDTH = rule(caseless_pipeline(['сотых', 'сотая'])).interpretation(const(10**-2))
TENTH = rule(caseless_pipeline(['десятых', 'десятая'])).interpretation(const(10**-1))

THOUSAND = or_(
    rule(caseless('т'), DOT),
    rule(caseless('тыс'), DOT.optional()),
    rule(normalized('тысяча')),
    rule(normalized('тыща')),
).interpretation(const(10**3))

MILLION = or_(
    rule(caseless('млн'), DOT.optional()),
    rule(normalized('миллион'))
).interpretation(const(10**6))

MILLIARD = or_(
    rule(caseless('млрд'), DOT.optional()),
    rule(normalized('миллиард'))
).interpretation(const(10**9))

TRILLION = or_(
    rule(caseless('трлн'), DOT.optional()),
    rule(normalized('триллион'))
).interpretation(const(10**12))

MULTIPLIER = or_(
    THOUSANDTH,
    HUNDREDTH,
    TENTH,
    THOUSAND,
    MILLION,
    MILLIARD,
    TRILLION
).interpretation(Number.multiplier)

NUM_RAW = rule(morph_pipeline(NUMS_RAW).interpretation(Number.int.normalized().custom(NUMS_RAW.get)))
NUM_INT = rule(INT).interpretation(Number.int.custom(int))
NUM = or_(
    NUM_RAW,
    NUM_INT
).interpretation(Number.int)

NUMBER = or_(
    rule(NUM, MULTIPLIER.optional())
).interpretation(Number)

### Алгоритм преобразования слов в числа



In [58]:
from natasha.extractors import Extractor
from yargy.parser import Match

class NumberExtractor(Extractor):
    def __init__(self):
        super(NumberExtractor, self).__init__(NUMBER)

    def replace_groups(self, text):
        if text:
            start = 0
            # тут мы достаем все слова в тексте которые относятся к числам
            matches = list(self.parser.findall(text))
            groups = []
            group_matches = []

            # проходимся по словам-числам и формируем группы
            # группа - это просто последовательность слов-чисел,
            # которые идут друг за другом
            for i, match in enumerate(matches):
                if i == 0:
                    start = match.span.start

                if i == len(matches) - 1:
                    next_match = match
                else:
                    next_match = matches[i + 1]

                group_matches.append(match.fact)
                if text[match.span.stop: next_match.span.start].strip() or next_match == match:
                    groups.append((group_matches, start, match.span.stop))
                    group_matches = []
                    start = next_match.span.start

            new_text = []
            start = 0

            for group in groups:
                num = 0
                nums = []
                new_text.append(text[start : group[1]])

                if len(group[0]) == 1 and text[group[1]:group[2]] == 'тыщ':
                    new_text += text[group[1]:group[2]]
                    start = group[2]
                    continue

                # тут непосредственно сам алгоритм которые решает,
                # как мы будем мерджить группу в конкретное число
                # в отличии от алгоритма из библиотеки мой выдает гораздо выше
                # качество, на тренировочных и тестовых данных
                # а так мой алгоритм не использует оператор += для строк, тк
                # это не эффективно
                stack = []
                for match in group[0]:
                  if not stack:
                    stack.append(match.int * match.multiplier if match.multiplier else match.int)
                  else:
                    top = stack[-1]
                    cur_num = match.int
                    if len(str(top)) > len(str(cur_num)):
                      top = stack.pop()
                      stack.append((top + cur_num) * (1 if match.multiplier is None else match.multiplier))
                    else:
                      stack.append(cur_num * match.multiplier if match.multiplier is not None else cur_num)

                new_text.append(' '.join(map(str, stack)))
                start = group[2]

            new_text.append(text[start:])

            if start == 0:
                return text
            else:
                return ''.join(new_text)
        else:
            return None

### Работа алгоритма на тренировочных данных и анализ ошибок

In [38]:
data = pd.read_feather("calibration.f")

In [39]:
extractor = NumberExtractor()
data['answer'] = data['task_text'].apply(extractor.replace_groups)

In [40]:
wrong_answers = list(data[data['answer'] != data['ground_truth']].index)

In [41]:
len(wrong_answers)

4

#### Тут в ответе не преобразовывается слово 'дваста' в 200, хотя в данных есть другой пример со словом 'дваста', которое уже преобразуется в ответе в число 200, поэтому не было смысла хардкодить этот кейс

In [42]:
data['task_text'].loc[wrong_answers[0]]

'дваста рублей скидка на товар если сделать заказ до трёх часов дня тогда цена станет ниже и покупатель захочет приобрести продукт потому что акция действует только ограниченное время'

In [43]:
data['ground_truth'].loc[wrong_answers[0]]

'дваста рублей скидка на товар если сделать заказ до 3 часов дня тогда цена станет ниже и покупатель захочет приобрести продукт потому что акция действует только ограниченное время'

In [44]:
data['answer'].loc[wrong_answers[0]]

'200 рублей скидка на товар если сделать заказ до 3 часов дня тогда цена станет ниже и покупатель захочет приобрести продукт потому что акция действует только ограниченное время'

#### Тут довольно понятный случай, у нас два числа одинакового порядка, но как я не пытался его учесть в алгоритме обязательно ломалось что то другое и качество довольно сильно падало

In [45]:
data['task_text'].loc[wrong_answers[1]]

'да ну потому что опять же разобрать там десять минут как конкурирующие компании продают это не составляет труда если ну ладно я полностью поддерживаю вас по поводу того что действительно существуют такие люди как вы описали да там перекупщики ну ладно но это не все продавцы ну то есть у нас семьдесят миллионов два миллиона действующих клиентов да то есть вряд ли все конкуренты будут какими то странными посредниками ну точно же есть ну я могу вам еще пару примеров отправить если вам этого не хватает да то есть в точности'

In [46]:
data['ground_truth'].loc[wrong_answers[1]]

'да ну потому что опять же разобрать там 10 минут как конкурирующие компании продают это не составляет труда если ну ладно я полностью поддерживаю вас по поводу того что действительно существуют такие люди как вы описали да там перекупщики ну ладно но это не все продавцы ну то есть у нас 70000000 2000000 действующих клиентов да то есть вряд ли все конкуренты будут какими то странными посредниками ну точно же есть ну я могу вам еще пару примеров отправить если вам этого не хватает да то есть в точности'

In [47]:
data['answer'].loc[wrong_answers[1]]

'да ну потому что опять же разобрать там 10 минут как конкурирующие компании продают это не составляет труда если ну ладно я полностью поддерживаю вас по поводу того что действительно существуют такие люди как вы описали да там перекупщики ну ладно но это не все продавцы ну то есть у нас 70000002000000 действующих клиентов да то есть вряд ли все конкуренты будут какими то странными посредниками ну точно же есть ну я могу вам еще пару примеров отправить если вам этого не хватает да то есть в точности'

#### Тут довольно сложный кейс, не понятно как это правильно обработать используя мой алгоритм так чтобы не просело качество в других кейсах

In [51]:
data['task_text'].loc[wrong_answers[2]]

'я просто сейчас посмотрела одно из ваших объявлений и подумала какова цена будет за двести два два'

In [52]:
data['ground_truth'].loc[wrong_answers[2]]

'я просто сейчас посмотрела 1 из ваших объявлений и подумала какова цена будет за 200 2 2'

In [53]:
data['answer'].loc[wrong_answers[2]]

'я просто сейчас посмотрела 1 из ваших объявлений и подумала какова цена будет за 204'

#### Тут в моем понимании ответ неправильный, хотя мой тоже не идеальный, в идеале наверное хотелось видеть ответ вида: 200000 350000, а не 200 350000

In [54]:
data['task_text'].loc[wrong_answers[3]]

'хорошо давайте тогда отправлю уведомление в колокольчик с этой суммой по вашей теме чтобы понимать среднюю стартовую сумму минимальную на что можно ориентироваться и что брать за основу поэтому начинайте расходовать с дня после завтра если возникнут какие то вопросы обращайтесь пожалуйста по мере свободного времени пообщаюсь с вами кстати любую итоговую сумму я пришлю завтра или послезавтра в зависимости от загруженности двеси триста пятьдесят тысяч в среднем по вашей категории ну или около того на что можно ориентироваться при планировании расходов и бюджета в целом да и вообще если будут какие то вопросы по поводу этих данных обращайтесь в любое время я постараюсь ответить'

In [55]:
data['ground_truth'].loc[wrong_answers[3]]

'хорошо давайте тогда отправлю уведомление в колокольчик с этой суммой по вашей теме чтобы понимать среднюю стартовую сумму минимальную на что можно ориентироваться и что брать за основу поэтому начинайте расходовать с дня после завтра если возникнут какие то вопросы обращайтесь пожалуйста по мере свободного времени пообщаюсь с вами кстати любую итоговую сумму я пришлю завтра или послезавтра в зависимости от загруженности 235000 в среднем по вашей категории ну или около того на что можно ориентироваться при планировании расходов и бюджета в целом да и вообще если будут какие то вопросы по поводу этих данных обращайтесь в любое время я постараюсь ответить'

In [56]:
data['answer'].loc[wrong_answers[3]]

'хорошо давайте тогда отправлю уведомление в колокольчик с этой суммой по вашей теме чтобы понимать среднюю стартовую сумму минимальную на что можно ориентироваться и что брать за основу поэтому начинайте расходовать с дня после завтра если возникнут какие то вопросы обращайтесь пожалуйста по мере свободного времени пообщаюсь с вами кстати любую итоговую сумму я пришлю завтра или послезавтра в зависимости от загруженности 200 350000 в среднем по вашей категории ну или около того на что можно ориентироваться при планировании расходов и бюджета в целом да и вообще если будут какие то вопросы по поводу этих данных обращайтесь в любое время я постараюсь ответить'

### Формирование ответа

In [57]:
source_data = pd.read_feather("test.f")
source_data["answer"] = source_data["task_text"].apply(extractor.replace_groups)
source_data.to_csv("answer.csv", index=False)